In [1]:
import graphviz
import numpy as np
import subprocess
import pandas as pd

In [8]:

def create_visualization(input_mat, input_node_labels, file_name_str, node_size=0.1, edge_num_to_show=100):
    np.fill_diagonal(input_mat, 0) # remove self-loops
    # further thresholding: only keep the top edge_num_to_show edges
    flat = input_mat.flatten()
    flat.sort()
    threshold = flat[-edge_num_to_show]
    input_mat[input_mat<threshold] = 0
    input_mat_min = threshold
    input_mat_max = input_mat.max()
    graph_matrix = (input_mat - input_mat_min)/(input_mat_max - input_mat_min)
    graph_matrix[graph_matrix<0] = 0
    # input_nlabel_max = input_node_labels.max()
    # input_nlabel_min = input_node_labels.min()
    print(input_node_labels)
    input_nlabel_max = max(input_node_labels)
    input_nlabel_min = min(input_node_labels)
    node_labels = (input_node_labels - input_nlabel_min)/(input_nlabel_max - input_nlabel_min)
    def float_to_color(a:float):
        # theta = a * np.pi / 2
        # return cmath.sin(theta), cmath.cos(theta), 1.0
        # use HSV color instead
        a = a/2
        return "{:.3f}".format(a) + " 1.000 1.000"
    def float_to_color2(a:float):
        # theta = a * np.pi / 2
        # return "{:.3f} {:.3f}".format(cmath.sin(theta), cmath.cos(theta)) + " 1.0"
        # use HSV color instead
        a = (a + 1)/2
        return "{:.3f}".format(a) + " 1.000 0.6"
    g = graphviz.Digraph('G', filename='hello.gv')
    # globa attributes
    g.attr(scale='3')
    # create nodes
    for node_idx, node_label in enumerate(node_labels):
        color_rgb = float_to_color(node_label)
        # set the attribute of the current node
        g.attr('node', label='', style='filled', shape='circle', fontsize='5', margin="0,0", width=str(node_size), height=str(node_size))
        # g.attr('node', shape='circle', fontsize='5', margin="0,0", width="0.1", height="0.1")
        g.attr('node', color=color_rgb)
        # create node
        # g.node(str(node_idx), str(node_label))
        g.node(str(node_idx))
    # create edges
    for (idx_start,idx_end), val in np.ndenumerate(graph_matrix):
        #  set the attribute of the current edge (mainly to alter the edge width)
        # g.attr(size='0.5')
        # create edge
        # g.edge(str(idx_start),str(idx_end), label=str(val))
        if val > 0:
            color_rgb2 = float_to_color2(val)
            g.attr('edge', arrowsize='0.05', penwidth=str(0.05*np.exp(1+val)), color=color_rgb2)
            g.edge(str(idx_start),str(idx_end))
    with open(file_name_str+".dot", 'w') as f:
        f.write(g.source)
    subprocess.run(["neato -Tpdf "+file_name_str+".dot"+" > "+str(file_name_str)+".pdf"], check=True, shell=True)

In [3]:
import pickle as pk
out_path = '/home/sv496/project/metabolic_graph/data/E.coli_data/'

with open(out_path+ 'aug_NN_features.pk', 'rb') as b:
    graph_obj = pk.load(b)

S_matrix = pd.read_csv(out_path + 'S_matrix.csv', index_col = 0)

In [4]:
S_bool = S_matrix[S_matrix == 0].fillna(1).to_numpy()
RAG = np.matmul(np.transpose(S_bool), S_bool)
# RAG = pd.DataFrame(RAG, index = S_matrix.columns, columns = S_matrix.columns)

In [6]:
bounds = graph_obj['bounds']
solutions = graph_obj['solutions_mwclique']

In [10]:
create_visualization(RAG, solutions[0], 'ecoli_graph')

[  9.16716044   4.01914797   9.16716044 -18.33432088  -0.
 -12.81597346  -0.         -18.33432088  -0.         -12.81597346
  -0.          -2.88734364   0.          -0.          -0.
   8.39        -0.          -2.88734364   0.          -0.
   0.           4.39363052   2.88734364   9.16716044   0.
  -0.         -11.68416914  -0.           0.          -0.
   0.           1.75553932  -2.63100377  18.33432088  -0.
 -12.81597346   1.60139578  -0.          -0.           0.
  -0.          -0.           9.16716044   2.88734364   0.
   0.          11.68416914  12.81597346   4.01914797   0.
   0.          -9.16716044   0.           0.           9.53749538
  -1.13180432   2.63100377   0.          -0.          -0.
  -0.           0.           0.           9.16716044   0.
   0.          -4.01914797   1.60139578  -0.          -0.
  -0.          -0.          18.33432088   9.16716044  17.22529504
  -0.          17.22529504   0.          17.22529504  -0.
   0.           1.13180432  -0.          -0.    